<a href="https://colab.research.google.com/github/harbour22/darkcyan-tools/blob/main/DarkCyan-yolo-trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## General Imports
from google.colab import drive # type: ignore
from IPython.display import clear_output # type: ignore
from pathlib import Path
import json
import shutil
import os
import time
from datetime import datetime


In [2]:
!pip install --upgrade git+https://github.com/harbour22/darkcyan-tools.git
clear_output(wait=False)

In [3]:
!pip install ultralytics
clear_output(wait=False)

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
## App Imports
from darkcyan.darkcyan_training_utils import get_training_zip_name
from darkcyan.darkcyan_training_utils import save_config
from darkcyan.constants import GOOGLEDRIVE_DATA_ROOT, \
                               DEFAULT_GOOGLEDRIVE_YOLO_DATA_DIR, \
                               DEFAULT_GOOGLEDRIVE_YOLO_CONFIG_DIR, \
                               DEFAULT_GOOGLEDRIVE_YOLO_TRAINING_OUTPUT_DIR, \
                               DEFAULT_YOLO_TRAINING_CONFIG, \
                               DEFAULT_GOOGLEDRIVE_YOLO_ENGINE_DIR, \
                               YOLOMODELMAP, \
                               YoloBaseModels, \
                               DataType, DEFAULT_DET_TRAINING_YAML

from ultralytics import YOLO                               

In [6]:
## Load config
config_file = Path(GOOGLEDRIVE_DATA_ROOT) / \
                   DEFAULT_GOOGLEDRIVE_YOLO_CONFIG_DIR / \
                   DEFAULT_YOLO_TRAINING_CONFIG

with open(config_file, "r", encoding="utf-8") as f:
    config = json.load(f)
config

{'version': '4.1',
 'type': 'det',
 'training_data': 'limetree_v4.1_det.zip',
 'basemodel': 'large',
 'epochs': 300,
 'imgsz': 640,
 'batchsize': 24,
 'creation_time': '2023-05-06 15:16:04',
 'created_by': 'chris'}

In [17]:
data_type = DataType[config['type']]
model_size = YoloBaseModels[config['basemodel']]
batch = config['batchsize']
epochs = config['epochs']

project_path = Path(GOOGLEDRIVE_DATA_ROOT) / \
               DEFAULT_GOOGLEDRIVE_YOLO_TRAINING_OUTPUT_DIR / \
               config['type'] / f"darkcyan_{config['version']}" / \
               model_size.name

data = data_path = Path(f"/content/{config['type']}_training_data")
imgsz = config['imgsz']

if data_type == DataType.det:
  data = data / DEFAULT_DET_TRAINING_YAML

In [18]:
zip_filename = Path(GOOGLEDRIVE_DATA_ROOT) / DEFAULT_GOOGLEDRIVE_YOLO_DATA_DIR / get_training_zip_name(config['version'], data_type, True)
shutil.unpack_archive(zip_filename,data_path)

In [19]:
last_run = project_path / 'train' / 'weights' / 'last.pt'
if last_run.exists():
  base_model = last_run
  resume=True
else: 
  base_model = YOLOMODELMAP[data_type][model_size]
  resume=False
        
print(base_model)

model = YOLO(base_model) # pass any model type
start_time = time.time()
model.train(epochs=epochs, resume=resume, project=project_path.as_posix(), batch=batch, data=data.as_posix(), imgsz=imgsz, exist_ok = True)
end_time = time.time()
clear_output(wait=False)

Ultralytics YOLOv8.0.93 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


yolov8l.pt


yolo/engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/det_training_data/darkcyan.yaml, epochs=300, patience=50, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolo/training_output/det/darkcyan_4.1/large, name=None, exist_ok=True, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, works

KeyboardInterrupt: ignored

In [ ]:
engine_file_name = f"yolov8_{config['version']}_{config['basemodel']}-{config['type']}.pt"
config_file_name = f"yolov8_{config['version']}_{config['basemodel']}-{config['type']}.json"
training_output = project_path / 'train' / 'weights' / 'best.pt'
engine_output = Path(GOOGLEDRIVE_DATA_ROOT) / DEFAULT_GOOGLEDRIVE_YOLO_ENGINE_DIR / engine_file_name
config_output = Path(GOOGLEDRIVE_DATA_ROOT) / DEFAULT_GOOGLEDRIVE_YOLO_ENGINE_DIR / config_file_name

shutil.copy(training_output, engine_output)

config['output_engine'] = engine_file_name
config['colab_version'] = os.environ['COLAB_RELEASE_TAG']
config['elapsed_training_time_mins'] = f'{(end_time - start_time)/60:.2f}'
config["training_end_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


save_config(config, config_output)
